In [1]:
%pip install "ultralytics<=8.3.40" supervision roboflow
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.6/112.6 GB disk)


In [2]:
from ultralytics import YOLO
from IPython.display import display, Image

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("").project("")
version = project.version()
dataset = version.download("")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to VegMining-5 in yolov9:: 100%|██████████| 9674/9674 [00:01<00:00, 8284.26it/s] 


In [4]:

data_dir = dataset.location  # Automatically set to downloaded folder
print("📁 Dataset downloaded to:", data_dir)

📁 Dataset downloaded to: /content/VegMining-5


In [5]:
import os

# Change this path to your preferred Drive folder
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/yolo_outputs/v9"
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
print("v9 outputs will be saved here:", DRIVE_OUTPUT_DIR)


v9 outputs will be saved here: /content/drive/MyDrive/yolo_outputs/v9


In [6]:
!yolo task=detect mode=train model=yolov9s.pt data={dataset.location}/data.yaml epochs=50 imgsz=640 plots=True

100% 14.7M/14.7M [00:00<00:00, 158MB/s]
New https://pypi.org/project/ultralytics/8.3.214 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov9s.pt, data=/content/VegMining-5/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask